In [1]:
from network_utils_hybrid import *
import random
import numpy as np
import time
from docplex.mp.model import Model



In [27]:
num_qpus = 4
num_racks = 6
num_ToR = 2 # number of qpus per rack
F_inter = 0.85
F_intra = 0.9

epr = np.random.randint(0,10,size=(num_qpus,num_qpus))

mdl = Model(name='routing')
x = mdl.binary_var_matrix(num_qpus, num_racks, name="x")

# one-hot vector constraint
for i in range(num_qpus):
    mdl.add_constraint(mdl.sum(x[i, k] for k in range(num_racks)) == 1, f"qpu_{i}")

# At most num_ToR qpus per rack
for k in range(num_racks):
    mdl.add_constraint(mdl.sum(x[i, k] for i in range(num_qpus)) <= num_ToR, f"rack_{j}")

# intermediate variable for links btw qpu[i] and qpu[j]
z = mdl.binary_var_cube(num_qpus, num_qpus, num_racks, name="z")
for i in range(num_qpus):
    for j in range(i+1, num_qpus):
        for k in range(num_racks):
            mdl.add_constraint(z[i,j,k] <= x[i,k] + x[j,k])
            mdl.add_constraint(z[i,j,k] >= x[i,k] - x[j,k])
            mdl.add_constraint(z[i,j,k] >= x[j,k] - x[i,k])
            mdl.add_constraint(z[i,j,k] <= 2 - (x[i,k] + x[j,k]))

obj = 0
for i in range(num_qpus):
    for j in range(i+1, num_qpus):
        obj += epr[i,j] * np.log(F_inter) * mdl.sum(z[i, j, k] for k in range(num_racks))/2
        obj += epr[i,j] * np.log(F_intra) * (1-mdl.sum(z[i, j, k] for k in range(num_racks))/2)

mdl.maximize(obj)
mdl.solve()
# del solution
solution = mdl.solve()#log_output=True)

# solution_values = mdl.solution.get_values()
# print("Solution values:", solution_values)

# for i in range(num_qpus):
#         for k in range(num_racks):
#             print(solution[x[i,k]])
x_opt = np.array([[solution[x[i,j]] for i in range(num_qpus)] for j in range(num_racks)])
# # y_opt = np.array([solution[y[i]] for i in range(num_paths["ir"]) ])
print(x_opt)
print(solution[obj])



[[1. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
-4.984348942298596
